# Model Deployment for Iris Classification

In this notebook, we'll deploy the best model we trained using Azure AutoML as a web service. We'll perform the following steps:
1. Connect to the Azure ML workspace
2. Load the trained model and necessary files
3. Define inference configuration
4. Define deployment configuration
5. Deploy the model
6. Test the deployed service

## 1. Connect to the Azure ML workspace and load necessary libraries

In [ ]:
from azureml.core import Workspace, Model, Run
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants

# Connect to your workspace
ws = Workspace.from_config()
print(f"Connected to workspace: {ws.name}")

# Get the latest run
experiment = Run.get_context().experiment
latest_run = list(experiment.get_runs())[0]
best_run = latest_run.get_best_child()
print(f"Best run ID: {best_run.id}")

## 2. Load the trained model and necessary files

In [ ]:
# Download the scoring file for the best model from the Azure ML run's output and save it locally
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

# Download the Conda environment file used by the best model and save it locally
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

# Create an Azure ML environment using the downloaded Conda specification
env = Environment.from_conda_specification(name='myenv', file_path='myenv.yml')

# Load the model
model = best_run.register_model(model_name="iris_classification_model", model_path='outputs/model.pkl')
print(f"Model registered: {model.name}, version: {model.version}")

## 3. Define inference configuration

In [ ]:
# Define the inference configuration using the scoring script and the environment
inference_config = InferenceConfig(entry_script='inference/score.py', environment=env)

## 4. Define deployment configuration

In [ ]:
# Set up the deployment configuration for an ACI web service with 1.5 CPU cores, 1 GB of memory, and a description
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1.5, memory_gb=1, description='Iris')

## 5. Deploy the model

In [ ]:
service_name = 'iris-classification-service'

service = Model.deploy(ws,
                       service_name,
                       [model],
                       inference_config,
                       aciconfig)

service.wait_for_deployment(True)
print(f"Service state: {service.state}")
print(f"Service URL: {service.scoring_uri}")

## 6. Test the deployed service

In [ ]:
import requests
import json

# Test data
test_data = json.dumps({'data': [[5.1, 3.5, 1.4, 0.2]]})

# Set the content type
headers = {'Content-Type': 'application/json'}

# Send the request
response = requests.post(service.scoring_uri, test_data, headers=headers)

print(f"Prediction: {response.json()}")

## Conclusion

We've successfully deployed our Iris classification model as a web service using Azure Container Instances. The service is now ready to accept HTTP requests and return predictions.